In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
import json

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [24]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

In [25]:
@tool
def complexTool(intArg: int, floatArg: float, dictArg: dict) -> int:
    """
        Do something complex with a complex tool
    """
    return intArg*floatArg

In [26]:
llmWithTools=llmOpenAI.bind_tools(tools=[complexTool])

In [27]:
question="Use Complex Tool. The args are 5, 2.1 and an empty dictionary. Don't forget dict_arg"

In [28]:
llmWithTools.invoke(input=question)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_W9GMoSt0Lb76m9mOZ6Qy2tSB', 'function': {'arguments': '{"intArg":5,"floatArg":2.1}', 'name': 'complexTool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 76, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bmn6egmKzLhgX4MBrh9BzrmoGfgxj', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--081e2e28-a815-477f-9e6b-a24cb07e5a21-0', tool_calls=[{'name': 'complexTool', 'args': {'intArg': 5, 'floatArg': 2.1}, 'id': 'call_W9GMoSt0Lb76m9mOZ6Qy2tSB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 76, 'output_tokens': 23, 'total_tokens': 99, 'input_token_details': {'audio': 0, 'cache_read':

In [29]:
chain=llmWithTools|RunnableLambda(lambda msg: msg.tool_calls[0]['args'])|complexTool

In [30]:
chain.invoke(input=question)

10.5

<h3> Try Except Tool Call</h3>

In [31]:
def tryExceptToolCall(toolArgs: dict, config=RunnableConfig) -> Runnable:
    try:
        return complexTool.invoke(
            input=toolArgs,
            config=RunnableConfig
        )
    except Exception as e:
        return f"{toolArgs}:{type(e)}:{e}"
        return f"Calling Tool with Arguments \n\n{toolArgs}\n\nraised the following error\n\n{type(e):e}"

In [32]:
chain=llmWithTools|RunnableLambda(lambda msg: msg.tool_calls[0]['args'])|tryExceptToolCall 

In [33]:
chain.invoke(input=question)

"{'intArg': 5, 'floatArg': 2.1}:<class 'TypeError'>:unbound method dict.items() needs an argument"

<h3> Falling to a Better Model (Fallbacks) </h3>

In [34]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
betterllm=ChatOpenAI(model="gpt-3.5-turbo")

In [35]:
llmWithTools=llm.bind_tools(tools=[complexTool])
betterllmWithTools=betterllm.bind_tools(tools=[complexTool])

In [37]:
chain = llmWithTools | (lambda msg: msg.tool_calls[0]["args"]) | complexTool
betterChain=betterllmWithTools | (lambda msg: msg.tool_calls[0]["args"]) | complexTool

In [38]:
chainWithFallback=chain.with_fallbacks(fallbacks=[betterChain])

In [39]:
chainWithFallback.invoke(input=question)

10.5